In [1]:
# begin exploration here!
import numpy as np
import pandas as pd
import scipy.stats as stats

# preprocessing
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler

# model selection
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, cross_val_score

from sklearn.metrics import (r2_score, mean_squared_error, accuracy_score, precision_score, recall_score, make_scorer,
                             f1_score, roc_auc_score, roc_curve, precision_recall_curve)

# models
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LogisticRegression
from datetime import date

In [2]:
data = pd.read_csv('data/train.csv',index_col= 'SalesID')
test = pd.read_csv('data/test.csv',index_col= 'SalesID')

C:\Users\jeramirez\AppData\Local\Temp\ipykernel_17212\772843367.py:1: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/train.csv',index_col= 'SalesID')


In [3]:
#Round 1 Selecciono las primeras columnas
y_train = data['SalePrice']
X_train = data[['MachineID','YearMade','MachineHoursCurrentMeter','UsageBand','ProductGroup','saledate']]
X_test = test[['MachineID','YearMade','MachineHoursCurrentMeter','UsageBand','ProductGroup','saledate']]

In [81]:
X_train.describe().T

,count,mean,std,min,25%,50%,75%,max
MachineID,401125.0,1.217903e+06,440991.954249,0.0,1088697.0,1279490.0,1468067.0,2486330.0
YearMade,401125.0,1.899157e+03,291.797469,1000.0,1985.0,1995.0,2000.0,2013.0
MachineHoursCurrentMeter,142765.0,3.457955e+03,27590.256413,0.0,0.0,0.0,3025.0,2483300.0


In [4]:
#Clean Year Made
def clean_YearMade(df, m):    
    median_by_PG = df[['ProductGroup','YearMade']][df.YearMade>1960].groupby('ProductGroup').median().round().astype(int)
    for pg,ym in median_by_PG.iterrows():
        df['YearMade'][(df.YearMade==1000) & (df.ProductGroup == pg)] = ym
    df['YearMade'] = df.YearMade.fillna(df.YearMade.mean())
    return(df['YearMade'])

In [5]:
def convert_saledate(df):
    return(pd.to_datetime(df.astype(str), format='%m/%d/%Y %H:%M', errors='coerce').map(date.toordinal))

In [6]:
#Funcion para limpiar las horas
def clean_Hours(df, m):
    #Llenando los nan
    df = df.fillna(m)
    return(df)

In [7]:
def clean_usageBand(df):
    usage = {'Low':3,'High':1,'Medium':2}
    return(df.map(usage).fillna(0))

In [8]:
X_train['YearMade'] = clean_YearMade(X_train[['ProductGroup','YearMade']], X_train.YearMade.mean())
X_test['YearMade'] = clean_YearMade(X_test[['ProductGroup','YearMade']], X_train.YearMade.mean())

C:\Users\jeramirez\AppData\Local\Temp\ipykernel_17212\530721590.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['YearMade'][(df.YearMade==1000) & (df.ProductGroup == pg)] = ym
C:\Users\jeramirez\AppData\Local\Temp\ipykernel_17212\530721590.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['YearMade'][(df.YearMade==1000) & (df.ProductGroup == pg)] = ym
C:\Users\jeramirez\AppData\Local\Temp\ipykernel_17212\530721590.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

In [9]:
X_train['saledate'] = convert_saledate(X_train['saledate'])
X_test['saledate'] = convert_saledate(X_test['saledate'])

C:\Users\jeramirez\AppData\Local\Temp\ipykernel_17212\3447429734.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['saledate'] = convert_saledate(X_train['saledate'])
C:\Users\jeramirez\AppData\Local\Temp\ipykernel_17212\3447429734.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['saledate'] = convert_saledate(X_test['saledate'])


In [10]:
X_train['MachineHoursCurrentMeter'] = clean_Hours(X_train['MachineHoursCurrentMeter'], X_train.MachineHoursCurrentMeter.mean())
X_test['MachineHoursCurrentMeter'] = clean_Hours(X_test['MachineHoursCurrentMeter'], X_train.MachineHoursCurrentMeter.mean())

C:\Users\jeramirez\AppData\Local\Temp\ipykernel_17212\4058190766.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['MachineHoursCurrentMeter'] = clean_Hours(X_train['MachineHoursCurrentMeter'], X_train.MachineHoursCurrentMeter.mean())
C:\Users\jeramirez\AppData\Local\Temp\ipykernel_17212\4058190766.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['MachineHoursCurrentMeter'] = clean_Hours(X_test['MachineHoursCurrentMeter'], X_train.MachineHoursCurrentMeter.mean())


In [11]:
print(X_test['UsageBand'])
X_train['UsageBand'] = clean_usageBand(X_train['UsageBand'])
X_test['UsageBand'] = clean_usageBand(X_test['UsageBand'])
print(X_test['UsageBand'])

SalesID
1222837       NaN
1222839    Medium
1222841    Medium
1222843       Low
1222845    Medium
            ...  
6333344       NaN
6333345       NaN
6333347       NaN
6333348       NaN
6333349       NaN
Name: UsageBand, Length: 11573, dtype: object
SalesID
1222837    0.0
1222839    2.0
1222841    2.0
1222843    3.0
1222845    2.0
          ... 
6333344    0.0
6333345    0.0
6333347    0.0
6333348    0.0
6333349    0.0
Name: UsageBand, Length: 11573, dtype: float64


C:\Users\jeramirez\AppData\Local\Temp\ipykernel_17212\2360994206.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['UsageBand'] = clean_usageBand(X_train['UsageBand'])
C:\Users\jeramirez\AppData\Local\Temp\ipykernel_17212\2360994206.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['UsageBand'] = clean_usageBand(X_test['UsageBand'])


In [90]:
X_train.describe().T

,count,mean,std,min,25%,50%,75%,max
MachineID,401125.0,1.217903e+06,440991.954249,0.0,1088697.0,1.279490e+06,1.468067e+06,2486330.0
YearMade,401125.0,1.993757e+03,9.211757,1919.0,1989.0,1.995000e+03,2.000000e+03,2013.0
MachineHoursCurrentMeter,401125.0,3.457955e+03,16459.845969,0.0,2318.0,3.457955e+03,3.457955e+03,2483300.0
UsageBand,401125.0,3.761022e-01,0.870291,0.0,0.0,0.000000e+00,0.000000e+00,3.0
saledate,401125.0,7.317953e+05,2102.982075,726119.0,730400.0,7.323930e+05,7.334900e+05,734501.0


In [91]:
X_test.describe().T

,count,mean,std,min,25%,50%,75%,max
MachineID,11573.0,1.651495e+06,652248.533150,150.0,1.067304e+06,1.862151e+06,2.270530e+06,2485252.0
YearMade,11573.0,1.999486e+03,7.632649,1953.0,1.997000e+03,2.001000e+03,2.005000e+03,2014.0
MachineHoursCurrentMeter,11573.0,4.286834e+03,4208.889065,0.0,3.457955e+03,3.457955e+03,3.457955e+03,89200.0
UsageBand,11573.0,8.001383e-01,1.164440,0.0,0.000000e+00,0.000000e+00,2.000000e+00,3.0
saledate,11573.0,7.345681e+05,26.735685,734503.0,7.345460e+05,7.345700e+05,7.345900e+05,734621.0


In [92]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401125 entries, 1139246 to 6333342
Data columns (total 6 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   MachineID                 401125 non-null  int64  
 1   YearMade                  401125 non-null  float64
 2   MachineHoursCurrentMeter  401125 non-null  float64
 3   UsageBand                 401125 non-null  float64
 4   ProductGroup              401125 non-null  object 
 5   saledate                  401125 non-null  int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 21.4+ MB


In [93]:
#Scoring method

In [94]:
def rmsle(actual, predictions):
    log_diff = np.log(predictions+1) - np.log(actual+1)
    return np.sqrt(np.mean(log_diff**2))

In [95]:
score = make_scorer(rmsle, greater_is_better = False)

In [96]:
y_train = data['SalePrice']
X_train = X_train[['YearMade','MachineHoursCurrentMeter','UsageBand','saledate']]
X_test = X_test[['YearMade','MachineHoursCurrentMeter','UsageBand','saledate']]

#Generacion de modelos

In [75]:
parameters = {}
model = LinearRegression()
gs = GridSearchCV(model, parameters, cv=5, n_jobs=-1, verbose=1, scoring=score)
gs.fit(X_train, y_train)
gs.best_score_, gs.best_params_

Fitting 5 folds for each of 1 candidates, totalling 5 fits


(-0.7384809896158925, {})

In [76]:
parameters = {'alpha': np.logspace(-5, 5)}
model = Ridge()
gs = GridSearchCV(model, parameters, cv=5, n_jobs=-1, verbose=1, scoring=score)
gs.fit(X_train, y_train)
gs.best_score_, gs.best_params_

Fitting 5 folds for each of 50 candidates, totalling 250 fits


(-0.7380243502312891, {'alpha': 62505.51925273976})

In [77]:
parameters = {'alpha': np.logspace(-5, 5)}
model = Lasso()
gs = GridSearchCV(model, parameters, cv=5, n_jobs=-1, verbose=1, scoring=score)
gs.fit(X_train, y_train)
gs.best_score_, gs.best_params_

Fitting 5 folds for each of 50 candidates, totalling 250 fits


(-0.7317778648450293, {'alpha': 9540.954763499964})

In [30]:
#scalar = StandardScaler()
#X_train = scalar.fit_transform(X_train)
#X_test = scalar.transform(X_test)

parameters = {'n_neighbors': range(2,50), 'metric': ['manhattan', 'euclidean']}
model = KNeighborsRegressor()
gs = GridSearchCV(model, parameters, cv=5, n_jobs=-1, verbose=1, scoring=score)
gs.fit(X_train, y_train)
gs.best_score_, gs.best_params_

Fitting 5 folds for each of 96 candidates, totalling 480 fits


(-0.7574593396459919, {'metric': 'manhattan', 'n_neighbors': 46})

#Final Model

In [117]:
final = Lasso(alpha=9540.954763499964)
final.fit(X_train, y_train)
y_hat = final.predict(X_test)
finalPred = pd.DataFrame({'SalesID':X_test.index,'SalePrice':y_hat}).set_index('SalesID')
finalPred.to_csv('data/finalPred.csv')

In [103]:
X_test

,YearMade,MachineHoursCurrentMeter,UsageBand,saledate
SalesID,,,,
1222837,1999.486351,0.000000,0.0,734507
1222839,2006.000000,4412.000000,2.0,734507
1222841,2000.000000,10127.000000,2.0,734507
1222843,1999.486351,4682.000000,3.0,734507
1222845,2002.000000,8150.000000,2.0,734506
...,...,...,...,...
6333344,2005.000000,3457.955353,0.0,734569
6333345,2005.000000,3457.955353,0.0,734530
6333347,2005.000000,3457.955353,0.0,734530
